## Laboratorio 2-3 AWS-hadoop-spark

### EDA Casos covid Colombia

In [1]:
from pyspark.sql.functions import col,sum

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
6,application_1633793432130_0009,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#Cargamos el dataframe leyendo los datos desde el S3 directamente
covidDF = spark.read.csv('s3://sidluzstella/raw/casos_covid/casos_covid_colombia/', header=True,inferSchema = True, mode="DROPMALFORMED")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#Vemos la primera filas del dataframe
covidDF.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(fecha reporte web='6/3/2020 0:00:00', ID de caso=1, Fecha de notificación='2/3/2020 0:00:00', Código DIVIPOLA departamento=11, Nombre departamento='BOGOTA', Código DIVIPOLA municipio=11001, Nombre municipio='BOGOTA', Edad=19, Unidad de medida de edad=1, Sexo='F', Tipo de contagio='Importado', Ubicación del caso='Casa', Estado='Leve', Código ISO del país=380, Nombre del país='ITALIA', Recuperado='Recuperado', Fecha de inicio de síntomas='27/2/2020 0:00:00', Fecha de muerte=None, Fecha de diagnóstico='6/3/2020 0:00:00', Fecha de recuperación='13/3/2020 0:00:00', Tipo de recuperación='PCR', Pertenencia étnica=6, Nombre del grupo étnico=None)

In [4]:
#Revisamos el esquema del dataframe
covidDF.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- fecha reporte web: string (nullable = true)
 |-- ID de caso: integer (nullable = true)
 |-- Fecha de notificación: string (nullable = true)
 |-- Código DIVIPOLA departamento: integer (nullable = true)
 |-- Nombre departamento: string (nullable = true)
 |-- Código DIVIPOLA municipio: integer (nullable = true)
 |-- Nombre municipio: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Unidad de medida de edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo de contagio: string (nullable = true)
 |-- Ubicación del caso: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- Código ISO del país: integer (nullable = true)
 |-- Nombre del país: string (nullable = true)
 |-- Recuperado: string (nullable = true)
 |-- Fecha de inicio de síntomas: string (nullable = true)
 |-- Fecha de muerte: string (nullable = true)
 |-- Fecha de diagnóstico: string (nullable = true)
 |-- Fecha de recuperación: string (nullable = true)
 |-- Tipo de r

In [5]:
#Revisamos cuantos registros tenemos
row_number = covidDF.count()
row_number

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4969131

In [6]:
#Revisamos cuantas columnas tenemos
column_number = len(covidDF.dtypes)
column_number

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

23

In [7]:
#Para fines del ejercicio tomamos un subconjunto de los datos
covidDF1 = covidDF.sort('ID de caso').limit(100000)
row_number = covidDF1.count()
row_number

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

100000

In [8]:
#Quedamos con las mismas columnas
column_number = len(covidDF1.dtypes)
column_number

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

23

In [9]:
#Revisemos las columnas del dataframe
covidDF1.describe().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+------------------+---------------------+----------------------------+-------------------+-------------------------+----------------+------------------+------------------------+------+----------------+------------------+---------+-------------------+---------------+----------+---------------------------+----------------+--------------------+---------------------+--------------------+------------------+-----------------------+
|summary|fecha reporte web|        ID de caso|Fecha de notificación|Código DIVIPOLA departamento|Nombre departamento|Código DIVIPOLA municipio|Nombre municipio|              Edad|Unidad de medida de edad|  Sexo|Tipo de contagio|Ubicación del caso|   Estado|Código ISO del país|Nombre del país|Recuperado|Fecha de inicio de síntomas| Fecha de muerte|Fecha de diagnóstico|Fecha de recuperación|Tipo de recuperación|Pertenencia étnica|Nombre del grupo étnico|
+-------+-----------------+------------------+---------------------+------------------

In [10]:
#Revisemos valores faltantes
covidDF1.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in covidDF1.columns)).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----+------------------------+----+----------------+------------------+------+-------------------+---------------+----------+---------------------------+---------------+--------------------+---------------------+--------------------+------------------+-----------------------+
|fecha reporte web|ID de caso|Fecha de notificación|Código DIVIPOLA departamento|Nombre departamento|Código DIVIPOLA municipio|Nombre municipio|Edad|Unidad de medida de edad|Sexo|Tipo de contagio|Ubicación del caso|Estado|Código ISO del país|Nombre del país|Recuperado|Fecha de inicio de síntomas|Fecha de muerte|Fecha de diagnóstico|Fecha de recuperación|Tipo de recuperación|Pertenencia étnica|Nombre del grupo étnico|
+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----+---------

Vemos que varias columnas tienes valores faltantes

### Revisemos algunas variables

In [11]:
covidDF1.describe(covidDF1.columns[3], covidDF1.columns[5], covidDF1.columns[7], covidDF1.columns[13], covidDF1.columns[21]).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------------------------+-------------------------+------------------+-------------------+------------------+
|summary|Código DIVIPOLA departamento|Código DIVIPOLA municipio|              Edad|Código ISO del país|Pertenencia étnica|
+-------+----------------------------+-------------------------+------------------+-------------------+------------------+
|  count|                      100000|                   100000|            100000|                912|            100000|
|   mean|                   2631.6288|              25327.34487|          39.30175|  596.3475877192982|           5.57697|
| stddev|           6172.660309006445|       25830.580807180402|18.420127848324064| 255.56508665655986|1.1828227296582021|
|    min|                           5|                     5001|                 1|                 32|                 1|
|    max|                       47001|                    99001|               104|                862|                 6|
+-------+-------

In [12]:
#Casos por edad
covidDF1.groupBy("Edad").count().sort("count",ascending=True).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+
|Edad|count|
+----+-----+
| 101|    1|
| 104|    1|
| 103|    2|
| 102|    2|
| 100|   10|
|  99|   11|
|  98|   12|
|  97|   14|
|  96|   23|
|  95|   32|
|  93|   51|
|  94|   53|
|  92|   70|
|  91|   91|
|  90|  109|
|  89|  117|
|  88|  146|
|  86|  182|
|  87|  184|
|  84|  213|
+----+-----+
only showing top 20 rows

In [13]:
#Casos por sexo
covidDF1.groupBy("Sexo").count().sort("count",ascending=True).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+
|Sexo|count|
+----+-----+
|   F|45902|
|   M|54098|
+----+-----+

In [14]:
#Al agrupar por Sexo, podemos ver que la media de la edad es similar, no se observan diferencias.
covidDF1.groupby('Sexo').agg({'Edad': 'mean'}).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------------+
|Sexo|        avg(Edad)|
+----+-----------------+
|   F|39.34373229924622|
|   M|39.26612813782395|
+----+-----------------+

In [15]:
#Casos por departamento, se muestran los departamentos con mas casos, las primeras 20 filas
covidDF1.groupBy(covidDF1.columns[4]).count().sort("count",ascending=False).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----+
|Nombre departamento|count|
+-------------------+-----+
|             BOGOTA|30016|
|       BARRANQUILLA|13065|
|          ATLANTICO|10994|
|              VALLE|10404|
|          CARTAGENA| 8333|
|          ANTIOQUIA| 4554|
|             NARIÑO| 3520|
|       CUNDINAMARCA| 2827|
|           AMAZONAS| 2317|
|              CHOCO| 1636|
|              SUCRE| 1317|
|               META| 1269|
|              CESAR| 1056|
|     STA MARTA D.E.| 1022|
|             TOLIMA| 1013|
|            BOLIVAR|  998|
|          MAGDALENA|  869|
|            CORDOBA|  807|
|          SANTANDER|  756|
|          RISARALDA|  545|
+-------------------+-----+
only showing top 20 rows

### Codificamos las variables categóricas 

In [24]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol='Estado', outputCol='Estado' + "Index")
covidDF2 = indexer.fit(covidDF1).transform(covidDF1)

indexer = StringIndexer(inputCol='Sexo', outputCol='Sexo' + "Index")
covidDF3 = indexer.fit(covidDF2).transform(covidDF2)

indexer = StringIndexer(inputCol=covidDF1.columns[10], outputCol='TipoDeContagio' + "Index")
covidDF4 = indexer.fit(covidDF3).transform(covidDF3)

indexer = StringIndexer(inputCol=covidDF1.columns[15], outputCol=covidDF1.columns[15] + "Index")
covidDF5 = indexer.fit(covidDF4).transform(covidDF4)

covidDF5.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----+------------------------+----+----------------+------------------+------+-------------------+---------------+----------+---------------------------+---------------+--------------------+---------------------+--------------------+------------------+-----------------------+-----------+---------+-------------------+---------------+
|fecha reporte web|ID de caso|Fecha de notificación|Código DIVIPOLA departamento|Nombre departamento|Código DIVIPOLA municipio|Nombre municipio|Edad|Unidad de medida de edad|Sexo|Tipo de contagio|Ubicación del caso|Estado|Código ISO del país|Nombre del país|Recuperado|Fecha de inicio de síntomas|Fecha de muerte|Fecha de diagnóstico|Fecha de recuperación|Tipo de recuperación|Pertenencia étnica|Nombre del grupo étnico|EstadoIndex|SexoIndex|TipoDeContagioIndex|RecuperadoIndex|
+-----------------+----------+------------

## Correlación entre variables

In [25]:
from pyspark.mllib.stat import Statistics

# select variables to check correlation
df_features = covidDF5.select(covidDF5.columns[3],"Edad",covidDF5.columns[21],'EstadoIndex','SexoIndex','TipoDeContagioIndex','RecuperadoIndex') 

# create RDD table for correlation calculation
rdd_table = df_features.rdd.map(lambda row: row[0:])

# get the correlation matrix
corr_mat=Statistics.corr(rdd_table, method="pearson")
print(corr_mat)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[[ 1.00000000e+00  2.74655555e-02  3.02410687e-02  1.88762296e-02
  -1.88084564e-03 -1.76382235e-01  2.47597429e-02]
 [ 2.74655555e-02  1.00000000e+00  1.78514654e-02  3.32824644e-01
   2.09942783e-03 -1.16811177e-01  3.28750345e-01]
 [ 3.02410687e-02  1.78514654e-02  1.00000000e+00  4.52124839e-03
   1.34421595e-04 -2.28287770e-02  3.97949806e-03]
 [ 1.88762296e-02  3.32824644e-01  4.52124839e-03  1.00000000e+00
  -3.13570104e-02 -8.78903567e-02  8.77083964e-01]
 [-1.88084564e-03  2.09942783e-03  1.34421595e-04 -3.13570104e-02
   1.00000000e+00 -2.28154373e-02 -3.36816459e-02]
 [-1.76382235e-01 -1.16811177e-01 -2.28287770e-02 -8.78903567e-02
  -2.28154373e-02  1.00000000e+00 -9.78788846e-02]
 [ 2.47597429e-02  3.28750345e-01  3.97949806e-03  8.77083964e-01
  -3.36816459e-02 -9.78788846e-02  1.00000000e+00]]

Solo encontramos fuerte correlación entre el estado y Recuperado

### Validar queries en SQL

In [29]:
from pyspark.ml.feature import SQLTransformer

#Consultemos los datos de las personas mayores de 100 años
sqlTrans = SQLTransformer(
    statement="SELECT * FROM __THIS__ where Edad > 100")
sqlTrans.transform(covidDF5).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----+------------------------+----+----------------+------------------+---------+-------------------+---------------+----------+---------------------------+-----------------+--------------------+---------------------+--------------------+------------------+-----------------------+-----------+---------+-------------------+---------------+
|fecha reporte web|ID de caso|Fecha de notificación|Código DIVIPOLA departamento|Nombre departamento|Código DIVIPOLA municipio|Nombre municipio|Edad|Unidad de medida de edad|Sexo|Tipo de contagio|Ubicación del caso|   Estado|Código ISO del país|Nombre del país|Recuperado|Fecha de inicio de síntomas|  Fecha de muerte|Fecha de diagnóstico|Fecha de recuperación|Tipo de recuperación|Pertenencia étnica|Nombre del grupo étnico|EstadoIndex|SexoIndex|TipoDeContagioIndex|RecuperadoIndex|
+-----------------+----------+--